In [36]:
import pandas as pd


data = pd.read_csv("data.csv")


data['year'] = pd.to_datetime(data['publication_date'], format='%d/%m/%Y').dt.year


data['subjectArea'] = data['subjectArea'].apply(lambda x: x.split(';'))


data = data.explode('subjectArea').explode('keywords')


keyword_trends = data.groupby(['year', 'subjectArea', 'keywords']).size().reset_index(name='count')


total_counts = data.groupby(['year', 'subjectArea']).size().reset_index(name='total_papers')
keyword_trends = keyword_trends.merge(total_counts, on=['year', 'subjectArea'])
keyword_trends['normalized_count'] = keyword_trends['count'] / keyword_trends['total_papers']


EMERGING_THRESHOLD = 0.95


latest_year = keyword_trends['year'].max()
latest_trends = keyword_trends[keyword_trends['year'] == latest_year]
latest_trends['is_emerging'] = (latest_trends['normalized_count'] > EMERGING_THRESHOLD).astype(int)


emerging_keywords = (
    latest_trends[latest_trends['is_emerging'] == 1]
    .groupby('subjectArea')['keywords']
    .apply(lambda x: '; '.join(sorted(set(x))))
    .reset_index()
)




C:\Users\VivoBook\AppData\Local\Temp\ipykernel_40888\4149906671.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_trends['is_emerging'] = (latest_trends['normalized_count'] > EMERGING_THRESHOLD).astype(int)


In [38]:


emerging_keywords.columns = ['Subject Area', 'Emerging Keywords']
emerging_keywords.to_csv("emerging_keywords_statistical.csv", index=False)
print(emerging_keywords)

            Subject Area                                  Emerging Keywords
0         AGRI,BUSI,ECON  Area yield,Crop insurance,Insurance product de...
1         AGRI,EART,ARTS  Anthropocene,Bang Khun Thian,Charcoal analysis...
2              AGRI,ENER  CaO/HZSM-5,Central composite rotatable design,...
3              AGRI,SOCI  Aroma components,Fermented soybean,Flavor,GC‒M...
4    AGRI,SOCI,ENVI,ECON  Death and disappearance,Fishing industry,Socia...
..                   ...                                                ...
114            SOCI,ENVI  Adaptive role,Complementarity,Entry barriers,M...
115            SOCI,HEAL  adolescent workers,occupational safety,Parasym...
116       VETE,AGRI,IMMU  anopheline mosquito,genetic structure,identifi...
117            VETE,IMMU  EEHV,Epitopes,Glycoprotein B,Peripheral blood ...
118            VETE,PHAR  Crocodylus porosus,estuarine crocodiles,intram...

[119 rows x 2 columns]
